<div style="background-color:#000;"><img src="pqn.png"></img></div>

### Define stock tickers and download historical data

Let's start by defining the stock tickers we want to analyze and specify the date range for our historical data. Then we download the adjusted closing prices for these stocks using the yfinance library.

In [ ]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = "2020-01-01"
end_date = "2024-12-31"

In [ ]:
stock_data = yf.download(
    tickers, 
    start=start_date, 
    end=end_date
)['Adj Close']

In [ ]:
port_returns = (
    stock_data
    .pct_change()
    .sum(axis=1)
)

In [ ]:
port_returns.name = "port_returns"

We define a list of stock tickers for Apple, Microsoft, and Google, specifying the time period from January 1, 2020, to December 31, 2024. Using yfinance, we download the adjusted closing prices for these stocks, which reflects the stock's actual market price adjusted for splits and dividends. We then compute the daily percentage change to calculate the portfolio's daily returns by summing up the individual returns of each stock. Finally, we name this series "port_returns" for clarity in subsequent analysis.

### Fetch Fama French factors and preprocess data

Now we will fetch the Fama French factors, which are widely used in finance to analyze and explain returns. We'll then align this data with our stock returns.

In [ ]:
fama_french = web.DataReader(
    'F-F_Research_Data_Factors_daily',
    'famafrench',
    start_date,
    end_date
)[0]

In [ ]:
fama_french = fama_french / 100  # Convert to decimals
fama_french.index = fama_french.index.tz_localize("UTC")

In [ ]:
data = fama_french.join(port_returns, how='inner')

In [ ]:
excess_returns = data.port_returns - data.RF

We retrieve daily Fama French factors using the pandas_datareader library, which provides the size, value, and market risk factors. These factors are in percentage form, so we convert them into decimals for accurate computations. The index of this data is then localized to UTC to match the timezone of our stock data. We join the Fama French data with our portfolio returns, keeping only the dates common to both datasets. Finally, we calculate the excess returns of the portfolio by subtracting the risk-free rate (RF) from the portfolio returns.

### Model excess returns using Fama French factors

Next, we prepare the independent variables for our regression model, add a constant term, and fit the model to the data. This will help us understand the influence of Fama French factors on our excess returns.

In [ ]:
X = data[['SMB', 'HML']]
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(excess_returns, X).fit()

In [ ]:
hedge_weights = -model.params[1:]

We select the Small-Minus-Big (SMB) and High-Minus-Low (HML) factors from our dataset as our independent variables. To account for the intercept in our regression model, we add a constant to this matrix. We then fit an Ordinary Least Squares (OLS) regression model using statsmodels to explain the excess returns of our portfolio by these factors. The resulting model's parameters (excluding the constant) are negated to calculate the hedge weights, indicating how much exposure to each factor is needed to minimize risk.

### Simulate and analyze the hedged portfolio

Finally, we use the hedge weights to create a hedged portfolio and simulate its returns. We compare these with the unhedged returns to analyze the impact of hedging.

In [ ]:
hedge_portfolio = (data[['SMB', 'HML']] @ hedge_weights).dropna()

In [ ]:
hedged_portfolio_returns = port_returns.loc[hedge_portfolio.index] + hedge_portfolio

In [ ]:
hedge = pd.DataFrame({
    "unhedged_returns": port_returns.loc[hedged_portfolio_returns.index],
    "hedged_returns": hedged_portfolio_returns
})

In [ ]:
hedge.mean() / hedge.std()

We calculate the hedged portfolio returns by applying the hedge weights to the SMB and HML factors, effectively offsetting their impact on the portfolio. We then add these hedged returns to the original portfolio returns, aligning them by date to ensure accuracy. We construct a DataFrame to hold both unhedged and hedged returns for comparison. Finally, we compute the Sharpe ratio for both sets of returns, which measures risk-adjusted performance. This helps us understand the effectiveness of our hedging strategy by comparing the consistency and risk of returns before and after hedging.

### Your next steps

Try experimenting with different stock tickers or adjusting the date range to see how the portfolio performance changes. You could also explore incorporating additional factors into the model to better capture the nuances of market behavior. Finally, this analysis uses the same data over the entire analysis period, which could introduce lookahead bias. In a few lines of code, you can create a rolling regression to avoid this.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.